[Home](http://realai.org/) > [Course](http://realai.org/course/) > [TensorFlow](http://realai.org/course/tensorflow/) > [Setting Up Your Computer](http://realai.org/course/tensorflow/#setting-up-your-computer) >

# TensorFlow Basics

*Last Updated: August 27, 2017*

TensorFlow is designed to work with large data sets. Let's first load some data into memory. The two lines below use a function from a sample project in the TensorFlow codebase to load the [MNIST database of handwritten digits](http://yann.lecun.com/exdb/mnist/), a popular data set for basic machine learning exercises.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets("MNIST_data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


According to the [source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/learn/python/learn/datasets/base.py), the function checks if the requested data already exist in the specified directory, and only downloads them from an online source when there is no local copy. If data are kept on the VM instance for all MNIST-related sessions, the output lines of "Successfully downloaded ..." should only be seen once.

The `input_data.read_data_sets()` function is introduced in an offical TensorFlow [tutorial](https://www.tensorflow.org/get_started/mnist/mechanics#download), which also includes a description of the training (MNIST.train), validation (MNIST.validation), and test (MNIST.test) data sets. They all come with a function called `next_batch()` that we can use to pull an arbitrary number of samples conveniently.

*Note: Getting data from the real world is typically much more tedious, and mostly independent of TensorFlow. [Here](http://nbviewer.jupyter.org/url/realai.org/course/jupyter/reading-MNIST-data.ipynb) is an example of downloading, parsing, then taking a look at the MNIST training data from the official source.*

In [2]:
batch = MNIST.train.next_batch(4)
batch[0].shape, batch[1].shape

((4, 784), (4, 10))

Each batch of N samples contains N images and N labels. Images are stored as 784(=28x28) numbers, and labels are stored as [one-hot](https://en.wikipedia.org/wiki/One-hot) vectors:

In [3]:
batch[1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

One-hot vectors are very intuitive, but images represented in numbers are not. Next let's use TensorFlow to visualize the images. As a software library, TensorFlow must be imported into Python before it can be used. It's customarily called `tf` to save typing:

In [4]:
import tensorflow as tf

The 784 numbers in each image need to be reshaped into 28x28 numbers, representing the intensities of the image's 28x28 pixels. We first construct a [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) to receive the input image, then use [tf.reshape](https://www.tensorflow.org/api_docs/python/tf/reshape) to convert it:

In [5]:
images = tf.placeholder(tf.float32, (None, 784), name="Images")
images_2d = tf.reshape(images, (-1, 28, 28, 1), name="Images_2D")

Usually we want some flexibility on the number of images in each batch, so the first dimension of the placeholder is set to `None`, which can be determined by TensorFlow when we feed actual data to the placeholder. It's reshaped to a 4D tensor. "-1" indicates that the first dimension of the 4D tensor is to be computed, since we intend it to be the number of images in a batch. The reshaped images are to be passed to [tf.summary.image](https://www.tensorflow.org/api_docs/python/tf/summary/image), a TensorBoard utility that requires an image to have 1, 3 or 4 channels. In our case the images have only 1 channel as they are in grayscale:

In [6]:
summ_images = tf.summary.image("Images_2D", images_2d, max_outputs=4)

Later in a TensorFlow session, tf.summary.image will evaluate into the data TensorBoard needs to display the images. We create a directory to store the data. The directory is placed under `/tmp` so that it'll be cleaned automatically:

In [7]:
LOGDIR = "/tmp/tf-basics/"
if tf.gfile.Exists(LOGDIR):
    tf.gfile.DeleteRecursively(LOGDIR)
tf.gfile.MkDir(LOGDIR)

FileWriter will write the image data into `LOGDIR`, and Session is where everything in TensorFlow is run with actual data. Additionally, [tf.get_default_graph](https://www.tensorflow.org/api_docs/python/tf/get_default_graph) will put the current computation graph on TensorBoard:

In [8]:
writer = tf.summary.FileWriter(LOGDIR, tf.get_default_graph())
sess = tf.Session()

When a [tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session) is run, input values are provided via a Python [dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) called `feed_dict`:

In [9]:
summ = sess.run(summ_images, feed_dict={images: batch[0]})
writer.add_summary(summ)

Close FileWriter and Session:

In [10]:
sess.close()
writer.close()

Type `tensorboard --logdir=/tmp/tf-basics/` in terminal to launch TensorBoard. It should display four digits in the *GRAPHS* section at `http://<External IP>:8888`, whose labels correspond to "Out[3]" above.

![](http://realai.org/course/tensorflow/tensorflow-basics-1.png)